Q2: How many lines are in the created metadata.yaml file?

In [61]:
! echo -n "metadata.yaml lines: " & wc -l ./mlops/mlops/homework_03/metadata.yaml |awk '{print $1}'

metadata.yaml lines: 55


Q3: How many records did we load?

In [63]:
import requests
from io import BytesIO
from typing import List

import pandas as pd

def ingest_files(**kwargs) -> pd.DataFrame:
    dfs: List[pd.DataFrame] = []

    for year, months in [(2023, (3, 4))]:
        for i in range(*months):
            response = requests.get(
                'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_'
                f'{year}-{i:02d}.parquet'
            )

            if response.status_code != 200:
                raise Exception(response.text)

            df = pd.read_parquet(BytesIO(response.content))
            dfs.append(df)

    
    return pd.concat(dfs)

df = ingest_files()
print(df.shape[0])

3403766


Q4: What's the size of the result?

In [64]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')
print(f"{df.shape[0]:,}")

3,316,216
